In [130]:
import mistune
import bs4
import yaml

from src.core.models import Topic

In [ ]:
yaml.dump(data, ff, allow_unicode=True)

In [157]:
import json

In [160]:
yaml.dump(needs[0], allow_unicode=True, default_flow_style=False)

inf: wish_pursuit=0010 state=successful
name: ensure everyone realizes their true wishes



In [146]:
[_['name'] for _ in needs]

[{'inf': 'wish_pursuit=0010 state=successful',
  'name': 'ensure everyone realizes their true wishes'},
 {'inf': 'brain=x1h1 state=latest',
  'name': 'ensure everyone has latest organs and healthy'},
 {'inf': 'eyes=zoptix state=latest',
  'name': 'make best eyes that fits to the skull'},
 {'inf': 'blood=hh1o state=latest',
  'name': 'make blood that can carry most oxygen and CO<sub>2</sub>'},
 {'inf': 'liver=anix state=latest',
  'name': 'make sure we can metabolize complex molecules, and synthesize bile for digestion'},
 {'inf': 'kidney=anio state=latest',
  'name': 'make sure we can remove waste materials, and regulate water levels'},
 {'inf': 'stomach=z88b state=latest', 'name': 'make sure we can digest food'},
 {'name': 'ensure breathing is active',
  'service': 'name=lungs state=started enabled=yes'},
 {'name': 'ensure blood circulation is active',
  'service': 'name=heart state=started enabled=yes'},
 {'inf': 'concrete=x1h1 state=present',
  'name': 'ensure they are supplied with

In [145]:
[d['name'] for d in needs]

['ensure everyone realizes their true wishes',
 'ensure everyone has latest organs and healthy',
 'make best eyes that fits to the skull',
 'make blood that can carry most oxygen and CO<sub>2</sub>',
 'make sure we can metabolize complex molecules, and synthesize bile for digestion',
 'make sure we can remove waste materials, and regulate water levels',
 'make sure we can digest food',
 'ensure breathing is active',
 'ensure blood circulation is active',
 'ensure they are supplied with concrete type X123']

In [129]:
topic = Topic.objects.filter(type=1).first()

soup = bs4.BeautifulSoup(
    mistune.markdown(topic.body),
    'html.parser'
)

need_lists = soup.find_all('code', {'class': 'lang-yml'})

needs = []

for i, need_list in enumerate(need_lists):
    txt = need_list.text
    yml = yaml.load(txt)
    for block in yml:
        if 'needs' in block.keys():
            needs.extend(block['needs'])

needs

[{'inf': 'wish_pursuit=0010 state=successful',
  'name': 'ensure everyone realizes their true wishes'},
 {'inf': 'brain=x1h1 state=latest',
  'name': 'ensure everyone has latest organs and healthy'},
 {'inf': 'eyes=zoptix state=latest',
  'name': 'make best eyes that fits to the skull'},
 {'inf': 'blood=hh1o state=latest',
  'name': 'make blood that can carry most oxygen and CO<sub>2</sub>'},
 {'inf': 'liver=anix state=latest',
  'name': 'make sure we can metabolize complex molecules, and synthesize bile for digestion'},
 {'inf': 'kidney=anio state=latest',
  'name': 'make sure we can remove waste materials, and regulate water levels'},
 {'inf': 'stomach=z88b state=latest', 'name': 'make sure we can digest food'},
 {'name': 'ensure breathing is active',
  'service': 'name=lungs state=started enabled=yes'},
 {'name': 'ensure blood circulation is active',
  'service': 'name=heart state=started enabled=yes'},
 {'inf': 'concrete=x1h1 state=present',
  'name': 'ensure they are supplied with

In [10]:
Topic.objects.create(
    title='.:en:Some Need',
    body='.:en\nsome condition',
    owner=topic.owner,
    type=0
)

<Topic: [Task] .:nl:Some Need>

In [137]:
Topic.objects.all()[1]

<Topic: [Goal] .:en:Name>

In [138]:
t = _

In [139]:
t.type=0

In [140]:
t.save()